<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Model-by-Patrick-Poon" data-toc-modified-id="Final-Model-by-Patrick-Poon-1">Final Model by Patrick Poon</a></span></li><li><span><a href="#Research-Question-/-Hypothesis" data-toc-modified-id="Research-Question-/-Hypothesis-2">Research Question / Hypothesis</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-3">Load Data</a></span></li><li><span><a href="#Create-pipeline-for-different-data-types" data-toc-modified-id="Create-pipeline-for-different-data-types-4">Create pipeline for different data types</a></span></li><li><span><a href="#Fitting-ML-Models" data-toc-modified-id="Fitting-ML-Models-5">Fitting ML Models</a></span><ul class="toc-item"><li><span><a href="#Ridge-Regressor" data-toc-modified-id="Ridge-Regressor-5.1">Ridge Regressor</a></span></li><li><span><a href="#HuberRegressor" data-toc-modified-id="HuberRegressor-5.2">HuberRegressor</a></span></li><li><span><a href="#Second-Run-HuberRegressor" data-toc-modified-id="Second-Run-HuberRegressor-5.3">Second Run HuberRegressor</a></span></li><li><span><a href="#Ensemble-Bagging" data-toc-modified-id="Ensemble-Bagging-5.4">Ensemble Bagging</a></span></li><li><span><a href="#Ensemble---Random-Forest" data-toc-modified-id="Ensemble---Random-Forest-5.5">Ensemble - Random Forest</a></span></li><li><span><a href="#Question:-What-if-we-are-not-putting-any-hyperparameters-and-just-run-RandomForest-using-default-parameters?" data-toc-modified-id="Question:-What-if-we-are-not-putting-any-hyperparameters-and-just-run-RandomForest-using-default-parameters?-5.6">Question: What if we are not putting any hyperparameters and just run RandomForest using default parameters?</a></span></li></ul></li><li><span><a href="#Final-Model-Evaluation-on-test-set" data-toc-modified-id="Final-Model-Evaluation-on-test-set-6">Final Model Evaluation on test set</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-7">Evaluation Metric</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from   sklearn.compose            import *
from   sklearn.experimental       import enable_iterative_imputer
from   sklearn.impute             import *
from   sklearn.metrics            import balanced_accuracy_score # Evaluation metric 2.0 
from   sklearn.pipeline           import Pipeline
from   sklearn.preprocessing      import *
from   sklearn.linear_model    import *
from   sklearn.metrics         import mean_absolute_error
from sklearn.base import BaseEstimator
from   sklearn.tree import *
from   sklearn.ensemble import *
from sklearn.model_selection  import RandomizedSearchCV
from sklearn.model_selection  import GridSearchCV
from sklearn import set_config
from sklearn.model_selection import cross_val_score

Final Model by Patrick Poon
---

Research Question / Hypothesis
----

Based on artists and characteristics on songs, find the populartiy scores of the songs



Load Data
-----

In [2]:
X = pd.read_csv('../data_2010/X.csv')
y = pd.read_csv('../data_2010/y.csv')

In [3]:
X = X.drop(columns=['Unnamed: 0','id','release_date', 'artists','name'])
y = y.drop(columns=['Unnamed: 0'])
y = y.values.ravel()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

Create pipeline for different data types
----

In [7]:
col = X.columns.tolist()

In [8]:
onehot_cat = ['mode','explicit', 'key', 'year'] # one hot encode these features

In [9]:
con_cat = [i for i in col if i not in onehot_cat] # continuous data

In [10]:
# Create onehotencoding pipeline, impute most frequent feature if missing
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True))
])

In [11]:
# Standardize continous data, use nearest neighbors for missing features
con_pipe = Pipeline([
    ('SS', StandardScaler()),
    ('KNN', KNNImputer(n_neighbors=5))
])

In [12]:
# Using column transformer to pack all features together, and assign them as categorial and continous data
preprocessor = ColumnTransformer([
    ('categorical', cat_pipe, onehot_cat),
    ('continuous', con_pipe, con_cat)
])

Fitting ML Models
---

In [13]:
# define dummy estimator for testing regressors
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

Usually, Dummy Estimators are used for testing multiple regressors at once. However, each regressor is broken down one by one in this analysis. This is to avoid running into issues, so each test can escape out after it is done. 

### Ridge Regressor

For ridge regressor, the goal is to find the optimal generalization from the alpha to avoid issues with outliers. From multiple iterations, the range of 0.05 - 0.2 performs best. 

For the targets, a target regressor transformer is applied so the output distribution is more of a normal distribution. This is done for optimizing performances of linear models.

A randomized cross-validation search is used to look for the best parameter in the search space, which is the alphas in this scenario. For the metric, root mean squared errors (RMSE) and mean squared errors (MSE) are used initially. The largest (neg) MSE or RMSE will indicate the best parameters from the CV searches. From inspection, RMSE is used as evaluation metric because RMSE penalizes large errors more, which is what we want to avoid here 

In [14]:
# Create space of candidate learning algorithms and their hyperparameters
alphas = np.linspace(0.05, 0.2, 25) # From various runs, the range of 0.05 to 0.2 seems to perform the best
search_space = [{'regre': [Ridge()], # Actual Estimator
                 'regre__alpha': alphas}] ## Alpha determines how much genalization we want to introduce in the model

# Final pipe for cross validation; quantiletransformer has a number of 300 quantiles. This was decided from multiple runs.\
# Changing from 300 to 1000 shows little impact.
ridge_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('regre', TransformedTargetRegressor(regressor=DummyEstimator(),
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))]) 

# Randomly pick numbers from search space (in this case, alphas) to find the best hyperparameter that performs best
ridge_randcv = RandomizedSearchCV(estimator=ridge_final_pipe, 
                                 param_distributions=search_space, 
                                 n_iter=20,
                                 cv=5,
                                 scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'],
                                 n_jobs=-1,
                                 verbose=1,
                                 random_state=42,
                                 refit='neg_root_mean_squared_error')

In [15]:
set_config(display='diagram')

ridge_randcv

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore')),
                                                                                               ('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent'))]),
                                                                               ['mode',
                                                                                'explicit',
                                                                                'key',
                                                                                'year']),
                                                                              ('continuous',
                                                                               Pipeline(steps=[('SS',
                                                                                                StandardScaler()),
                                                                                               ('KNN',
                                                                                                KNNImp...
                   param_distributions=[{'regre': [Ridge()],
                                         'regre__alpha': array([0.05   , 0.05625, 0.0625 , 0.06875, 0.075  , 0.08125, 0.0875 ,
       0.09375, 0.1    , 0.10625, 0.1125 , 0.11875, 0.125  , 0.13125,
       0.1375 , 0.14375, 0.15   , 0.15625, 0.1625 , 0.16875, 0.175  ,
       0.18125, 0.1875 , 0.19375, 0.2    ])}],
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_squared_error'],
                   verbose=1)

In [16]:
ridge_randcv.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.3s finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore')),
                                                                                               ('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent'))]),
                                                                               ['mode',
                                                                                'explicit',
                                                                                'key',
                                                                                'year']),
                                                                              ('continuous',
                                                                               Pipeline(steps=[('SS',
                                                                                                StandardScaler()),
                                                                                               ('KNN',
                                                                                                KNNImp...
                   param_distributions=[{'regre': [Ridge(alpha=0.2)],
                                         'regre__alpha': array([0.05   , 0.05625, 0.0625 , 0.06875, 0.075  , 0.08125, 0.0875 ,
       0.09375, 0.1    , 0.10625, 0.1125 , 0.11875, 0.125  , 0.13125,
       0.1375 , 0.14375, 0.15   , 0.15625, 0.1625 , 0.16875, 0.175  ,
       0.18125, 0.1875 , 0.19375, 0.2    ])}],
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_squared_error'],
                   verbose=1)

In [17]:
### Inspecting all cross-validation results and choose best score (RMSE) and best params.
ridge_randcv.cv_results_

{'mean_fit_time': array([0.11691213, 0.12135634, 0.10807405, 0.09795938, 0.08713331,
        0.09153152, 0.0843576 , 0.0760396 , 0.09149022, 0.09423919,
        0.11304483, 0.10027204, 0.07236214, 0.07728844, 0.10233407,
        0.08321805, 0.08302345, 0.09682226, 0.10120916, 0.09311409]),
 'std_fit_time': array([0.01026476, 0.00647312, 0.01099062, 0.00583488, 0.00393667,
        0.0070495 , 0.00129407, 0.00450163, 0.01237163, 0.00423971,
        0.00994228, 0.00181404, 0.00847471, 0.01121074, 0.00302421,
        0.01699812, 0.00717003, 0.00744264, 0.00486178, 0.00722676]),
 'mean_score_time': array([0.02476311, 0.02506642, 0.02419362, 0.01805582, 0.01906667,
        0.01978865, 0.01528683, 0.0181602 , 0.0264122 , 0.02223821,
        0.02761493, 0.01632314, 0.013134  , 0.02291751, 0.01949053,
        0.01701741, 0.02088075, 0.02112212, 0.0197722 , 0.0146935 ]),
 'std_score_time': array([0.0046893 , 0.00165524, 0.00472777, 0.00182401, 0.00154856,
        0.00214748, 0.00183531, 0.001027

In [18]:
ridge_randcv.best_score_

-25.26018482519345

In [19]:
ridge_randcv.best_params_

{'regre__alpha': 0.2, 'regre': Ridge(alpha=0.2)}

### HuberRegressor

Similar setup as Ridge Regressor, randomized search cross validation is used to find the best hyperparameters for huber. For this run, two hyperaparemters is searched through: Alphas and Eps. 

Alphas - For genalization <br>
Eps - For robustness to outliers. The smaller it is, the more robust it is. <br>
Max_iter - this is set as 1500 in order for huber to converge to results. However, lbfgs can sometimes fail, but ignoring warnings. 

In [20]:
# Create space of candidate learning algorithms and their hyperparameters
alphas = [0.0001, 0.01, 0.1, 1, 10] ## Similar to Ridge, this is for genalization
eps = [1.1, 1.2, 1.3, 1.4, 1.5] ## This is for adjusting how robust HuberRegressor is to outliers. 
search_space = [{'regre': [HuberRegressor(max_iter=1500)], #using max_iter as 1500 for converging results
                 'regre__alpha': alphas,
                 'regre__epsilon': eps}] 

huber_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('regre', TransformedTargetRegressor(regressor=DummyEstimator(),
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])
huber_randcv = RandomizedSearchCV(estimator=huber_final_pipe, 
                                 param_distributions=search_space, 
                                 n_iter=10,
                                 cv=5,
                                 scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'],
                                 n_jobs=-1,
                                 verbose=1,
                                 random_state=42,
                                 refit='neg_root_mean_squared_error')

In [21]:
huber_randcv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.3s finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore')),
                                                                                               ('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent'))]),
                                                                               ['mode',
                                                                                'explicit',
                                                                                'key',
                                                                                'year']),
                                                                              ('continuous',
                                                                               Pipeline(steps=[('SS',
                                                                                                StandardScaler()),
                                                                                               ('KNN',
                                                                                                KNNImp...
                                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                                                                         output_distribution='normal')))]),
                   n_jobs=-1,
                   param_distributions=[{'regre': [HuberRegressor(alpha=10,
                                                                  epsilon=1.4,
                                                                  max_iter=1500)],
                                         'regre__alpha': [0.0001, 0.01, 0.1, 1,
                                                          10],
                                         'regre__epsilon': [1.1, 1.2, 1.3, 1.4,
                                                            1.5]}],
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_squared_error'],
                   verbose=1)

In [22]:
huber_randcv.best_params_

{'regre__epsilon': 1.4,
 'regre__alpha': 10,
 'regre': HuberRegressor(alpha=10, epsilon=1.4, max_iter=1500)}

In [23]:
huber_randcv.best_score_

-25.34422416553995

### Second Run HuberRegressor

A second fit of Huber Regressor is conducted to fit hyperparameters in a tighter search space. This CV has the same paramters as the first Huber analysis.

In [24]:
# Create space of candidate learning algorithms and their hyperparameters
alphas = np.linspace(0.0001, 0.001, 10) # Huber requires the genearlization to be lower than Ridge, 
                                        # trying range of 0.0001 to 0.001
eps = np.linspace(1.4, 1.6, 10) # This is for adjusting how robust HuberRegressor is to outliers. 
search_space = [{'regre': [HuberRegressor(max_iter=1500)], # using max_iter as 1500 for converging results
                 'regre__alpha': alphas,
                 'regre__epsilon': eps}] 

huber_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('regre', TransformedTargetRegressor(regressor=DummyEstimator(),
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])
huber_randcv2 = RandomizedSearchCV(estimator=huber_final_pipe, 
                                 param_distributions=search_space, 
                                 n_iter=10,
                                 cv=5,
                                 scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'],
                                 n_jobs=-1,
                                 verbose=1,
                                 random_state=42,
                                 refit='neg_root_mean_squared_error')

In [25]:
huber_randcv2.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    4.3s finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore')),
                                                                                               ('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent'))]),
                                                                               ['mode',
                                                                                'explicit',
                                                                                'key',
                                                                                'year']),
                                                                              ('continuous',
                                                                               Pipeline(steps=[('SS',
                                                                                                StandardScaler()),
                                                                                               ('KNN',
                                                                                                KNNImp...
                                                                  max_iter=1500)],
                                         'regre__alpha': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009, 0.001 ]),
                                         'regre__epsilon': array([1.4       , 1.42222222, 1.44444444, 1.46666667, 1.48888889,
       1.51111111, 1.53333333, 1.55555556, 1.57777778, 1.6       ])}],
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_squared_error'],
                   verbose=1)

In [26]:
huber_randcv2.best_params_

{'regre__epsilon': 1.6,
 'regre__alpha': 0.00039999999999999996,
 'regre': HuberRegressor(alpha=0.00039999999999999996, epsilon=1.6, max_iter=1500)}

In [27]:
huber_randcv2.best_score_

-25.319244438706132

### Ensemble Bagging

Next, we are moving onto ensemble methods. Specifically Bagging and Random Forests. <br>

For bagging regressors, only two hyperparameters are explored here: n_est and max_features. <br>

n_est is specifying the number of trees in the model. This is done to reduce overfitting like decision trees. <br>
max_features represents the number of features chosen per each tree. This is a way of bootstrapping to weaken the trees. By doing so, this allows better generalization and avoid overfitting.

In [28]:
n_est = [100, 150] # num of trees (base_estimators)
max_features = [0.8, 1.0] # features in base_estimators
search_space = [{'bag': [BaggingRegressor(n_jobs=-1)], # Actual Estimator
                 'bag__n_estimators': n_est,
                 'bag__max_features': max_features}]

bag_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('bag', TransformedTargetRegressor(regressor=DummyEstimator(),
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])                

bag_randcv = RandomizedSearchCV(estimator=bag_final_pipe, 
                                 param_distributions=search_space,
                                 cv=5,
                                 scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'],
                                 n_jobs=-1,
                                 verbose=1,
                                 random_state=42,
                                 refit='neg_root_mean_squared_error')

In [29]:
bag_randcv.fit(X_train, y_train)

/Users/pkpoon/opt/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:  1.0min remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.0min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore')),
                                                                                               ('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent'))]),
                                                                               ['mode',
                                                                                'explicit',
                                                                                'key',
                                                                                'year']),
                                                                              ('continuous',
                                                                               Pipeline(steps=[('SS',
                                                                                                StandardScaler()),
                                                                                               ('KNN',
                                                                                                KNNImp...
                                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                                                                         output_distribution='normal')))]),
                   n_jobs=-1,
                   param_distributions=[{'bag': [BaggingRegressor(n_estimators=150,
                                                                  n_jobs=-1)],
                                         'bag__max_features': [0.8, 1.0],
                                         'bag__n_estimators': [100, 150]}],
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_squared_error'],
                   verbose=1)

In [30]:
bag_randcv.best_score_

-20.405664835077722

### Ensemble - Random Forest

For Random Forests, a total of 4 hyperparameters are searched through: 
- n_est
- min_samples_split
- min_samples_leaf
- max_depth<br>

n_est is specifying the number of trees in the model. Similar as Bagging, this is done to reduce overfitting like decision trees. <br>
min_samples_split represents the minimal sample to split a node. Smaller value introduces less variance in prediction for this node. <br>
min_samples_leaf represents how many samples needed per each leaf. The leaf node won't be splitted if either left or right leaf child has less samples if splitted. 
max_depth controls how deep a tree can go. 
<br>

In [31]:
n_est = [100, 150]
min_samples_split = [2, 5]
min_samples_leaf = [2]
max_depth = [100, 150, 250]
search_space = [{'rf': [RandomForestRegressor(n_jobs=-1)], # Actual Estimator
                 'rf__n_estimators': n_est,
                 'rf__min_samples_split': min_samples_split,
                 'rf__min_samples_leaf': min_samples_leaf,
                 'rf__max_depth': max_depth}]

rf_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('rf', TransformedTargetRegressor(regressor=DummyEstimator(),
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])                

rf_randcv = RandomizedSearchCV(estimator=rf_final_pipe, 
                                 param_distributions=search_space, 
                                 n_iter=12,
                                 cv=5,
                                 scoring=['neg_root_mean_squared_error', 'neg_mean_squared_error'],
                                 n_jobs=-1,
                                 verbose=1,
                                 random_state=42,
                                 refit='neg_root_mean_squared_error')

In [32]:
rf_randcv.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.4min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore')),
                                                                                               ('imputer',
                                                                                                SimpleImputer(add_indicator=True,
                                                                                                              strategy='most_frequent'))]),
                                                                               ['mode',
                                                                                'explicit',
                                                                                'key',
                                                                                'year']),
                                                                              ('continuous',
                                                                               Pipeline(steps=[('SS',
                                                                                                StandardScaler()),
                                                                                               ('KNN',
                                                                                                KNNImp...
                   param_distributions=[{'rf': [RandomForestRegressor(max_depth=100,
                                                                      min_samples_leaf=2,
                                                                      n_estimators=150,
                                                                      n_jobs=-1)],
                                         'rf__max_depth': [100, 150, 250],
                                         'rf__min_samples_leaf': [2],
                                         'rf__min_samples_split': [2, 5],
                                         'rf__n_estimators': [100, 150]}],
                   random_state=42, refit='neg_root_mean_squared_error',
                   scoring=['neg_root_mean_squared_error',
                            'neg_mean_squared_error'],
                   verbose=1)

In [33]:
# rf_randcv.cv_results_

In [34]:
rf_randcv.best_score_

-20.440163347563175

In [35]:
rf_randcv.best_params_

{'rf__n_estimators': 150,
 'rf__min_samples_split': 2,
 'rf__min_samples_leaf': 2,
 'rf__max_depth': 100,
 'rf': RandomForestRegressor(max_depth=100, min_samples_leaf=2, n_estimators=150,
                       n_jobs=-1)}

### Question: What if we are not putting any hyperparameters and just run RandomForest using default parameters?

In [36]:
orig_rf_pipe = Pipeline([('preprocessor', preprocessor),
                ('rf', TransformedTargetRegressor(regressor=RandomForestRegressor(n_jobs=-1),
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])   

In [37]:
## Using the same cross validation scoring and cv = 5
scores = cross_val_score(orig_rf_pipe, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')

In [38]:
np.mean(scores) # So this is worst than with hyperparameters 

-30.051352455408306

Final Model Evaluation on test set
---

Here are the final results from all the CV runs

In [39]:
print(f'Ridge Regressor: {round(ridge_randcv.best_score_, 3)}')
print(f'Huber Regressor -1: {round(huber_randcv.best_score_, 3)}')
print(f'Huber Regressor -2: {round(huber_randcv2.best_score_, 3)}')
print(f'Bagging Regressor: {round(bag_randcv.best_score_, 3)}')
print(f'Random Forest Regressor: {round(rf_randcv.best_score_, 3)}')

Ridge Regressor: -25.26
Huber Regressor -1: -25.344
Huber Regressor -2: -25.319
Bagging Regressor: -20.406
Random Forest Regressor: -20.44


We are picking Bagging Ensemble as our final candidate to test the test results. We are using Mean Absolute Error (MAE) here because we would like to correlate that to the song popularity abosulte eorror

In [40]:
bag_randcv.best_params_

{'bag__n_estimators': 150,
 'bag__max_features': 1.0,
 'bag': BaggingRegressor(n_estimators=150, n_jobs=-1)}

In [41]:
model_final = BaggingRegressor(n_estimators=150, max_features = 1.0, n_jobs=-1)
model_final_pipe = Pipeline([('preprocessor', preprocessor),
                ('bag', TransformedTargetRegressor(regressor=model_final,
                                                         transformer=QuantileTransformer(n_quantiles=300,
                                                         output_distribution='normal')))])
model_final_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='most_frequent'))]),
                                                  ['mode', 'explicit', 'key',
                                                   'year']),
                                                 ('continuous',
                                                  Pipeline(steps=[('SS',
                                                                   StandardScaler()),
                                                                  ('KNN',
                                                                   KNNImputer())]),
                                                  ['acousticness',
                                                   'danceability',
                                                   'duration_ms', 'energy',
                                                   'instrumentalness',
                                                   'liveness', 'loudness',
                                                   'speechiness', 'tempo',
                                                   'valence'])])),
                ('bag',
                 TransformedTargetRegressor(regressor=BaggingRegressor(n_estimators=150,
                                                                       n_jobs=-1),
                                            transformer=QuantileTransformer(n_quantiles=300,
                                                                            output_distribution='normal')))])

In [42]:
# Prediction on test set
y_pred = model_final_pipe.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.6f}")

MAE: 17.270750


Evaluation Metric
----

Mean absolute error is used to determine how close the predicted popularity score of a song is to the actual populartiy. This metric can be explained as "this model can predict that this song will have a popularity of this number, with a potential of 17 error". Future work will include correlating to  the Billboard top 100 songs. For instance: Taking the existing song title and popularity and comparing to Billboard. With benchmark of these songs and the MAE, the model can predict potential ranking of a song in Billboard based on the song characteristics.